In [1]:
import json
import requests
import pandas as pd
import time
from tqdm import tqdm

In [2]:
with open('keys.json') as f:
  keysRAW = f.read()
keys = json.loads(keysRAW)

In [3]:
api_url = "https://na1.api.riotgames.com"
grandmasterLeagueEnd = "/tft/league/v1/grandmaster"
response = requests.get(f"{api_url}{grandmasterLeagueEnd}", headers={"X-Riot-Token": keys["private"]})
response.status_code

200

In [4]:
GRANDMASTER = response.json()
GRANDMASTER

{'tier': 'GRANDMASTER',
 'leagueId': '19edec7a-a457-36a3-897d-a2f68b4c42a4',
 'queue': 'RANKED_TFT',
 'name': "Irelia's Weaponmasters",
 'entries': [{'summonerId': 'kIy1qHo-rjmtPnYY0Dn4PzyRsDxMBBUAxyit5zEB1TSTc-7f6BwKBlKI9A',
   'summonerName': '9year1duNA',
   'leaguePoints': 535,
   'rank': 'I',
   'wins': 125,
   'losses': 85,
   'veteran': False,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': True},
  {'summonerId': 'GVKwjbUfODUlssrR4LIbMvCGy9YhspLXgdMgbldusV8pjbE',
   'summonerName': 'RNG Lil UZI',
   'leaguePoints': 623,
   'rank': 'I',
   'wins': 167,
   'losses': 154,
   'veteran': False,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': False},
  {'summonerId': 'JLEZdyISIwz2a6nojw5JlCAinHdZly2Kn7Z121NUeRNz0X4',
   'summonerName': 'BBBBeo',
   'leaguePoints': 375,
   'rank': 'I',
   'wins': 155,
   'losses': 128,
   'veteran': False,
   'inactive': False,
   'freshBlood': True,
   'hotStreak': False},
  {'summonerId': 'Iz3dGEHAIdeGvOPAwcU6iBz7TqtIe13E

In [5]:
summonerIDs = []
for summoner in GRANDMASTER["entries"]:
  summonerIDs.append(summoner["summonerId"])

summonerIDs

['kIy1qHo-rjmtPnYY0Dn4PzyRsDxMBBUAxyit5zEB1TSTc-7f6BwKBlKI9A',
 'GVKwjbUfODUlssrR4LIbMvCGy9YhspLXgdMgbldusV8pjbE',
 'JLEZdyISIwz2a6nojw5JlCAinHdZly2Kn7Z121NUeRNz0X4',
 'Iz3dGEHAIdeGvOPAwcU6iBz7TqtIe13E88PnJEXFKWWkxTw',
 'Bm-RPDNNg3dINGc0nxKOo2YMhDBcHEv3nSY0YZp7cAfETI0',
 '4Xz_yRkZw31l9XJGmGLcC5R3gQE_4GWMena7-HLfPTGk184',
 'muQO49lzXihiMFVkux7CFTcBQhtlYSSQ7Aj1-wohUPCH_xjp',
 'R7O6_1-joKVvGC0I7yb2bv1QCx3nDGN57XEhdiHDgJ0w0lA',
 'cbvfKUodjTEkWDhHquPj5Jg1uDqDehJMvrPFC0fKuU--2ECf',
 'HypnK1g4PhxdoYolNapZSiNLOGM1XDsvzUlC6afI4LXCbR_f',
 'fom9dmzkge8r9BpfDs5_tVZv1dsiyBpQ22cS30vCaAriT-k',
 '4jzxfjsCLt7KI9KYSdaCk_yGdqdZb4UfTNMtSykccO2XjgU',
 '-T1WP3YFy0ZAjeE2UMXN0pi6jQSER5Tmqt2GX3p6Z1TUIdo',
 'JAK9EHKMsmnXtN2PkNSqx99r3LDEs0ql2cCXZ1uIMrlKk9c',
 '6BkR3MdEG8jYnVn7bL1eSV7eyCFjrQ_7cfR_V3zSlvN7UI0',
 '3u2eeRmQY1AcMkYVzQE08lXczl7ZpNjUqiAjZAV1WMi7bf_X',
 'Od1xOgYCo6MDjkwt2LXo2D23pUlgVcwvV3990aB-ZQsbNdKwTVlBKQzpSg',
 'fQaR5X3jDkXk78t9bfLKX_Q70fSLO7VOyXne62yaRY7A9cjr',
 'mz_sm5q7t6I7ir6Xy6wFv-Elc_an2zY-TA3

In [6]:
def gatherSummonerData (summonerID):
  calls = 0

  summonerEnd = "/tft/summoner/v1/summoners/"
  puuidsResponse = requests.get(f"{api_url}{summonerEnd}{summonerID}", headers={"X-Riot-Token": keys["private"]})
  calls += 1
  puuid = puuidsResponse.json()["puuid"]

  api_url_region_america = "https://americas.api.riotgames.com"
  matchPUUIDEnd = f"/tft/match/v1/matches/by-puuid/{puuid}/ids"
  start = 0
  count = 20
  matchesResponse = requests.get(f"{api_url_region_america}{matchPUUIDEnd}?start={start}&count={count}", headers={"X-Riot-Token": keys["private"]})
  calls += 1

  matches = matchesResponse.json()

  sleepTime = 0
  summonerMatchData = []
  for match in matches:
    matchEND = "/tft/match/v1/matches/"
    matchDataResponse = requests.get(f"{api_url_region_america}{matchEND}{match}", headers={"X-Riot-Token": keys["private"]})

    calls += 1
    time.sleep(0.1)
    sleepTime += 1

    matchData = {}
    for participant in matchDataResponse.json()["info"]["participants"]:
      matchData[participant["puuid"]] = {}
      matchData[participant["puuid"]]["placement"] = participant["placement"]
      matchData[participant["puuid"]]["traits"] = []
      matchData[participant["puuid"]]["units"] = []
      matchData[participant["puuid"]]["augments"] = []
      for trait in participant["traits"]:
        matchData[participant["puuid"]]["traits"].append(trait["name"])
      for unit in participant["units"]:
         matchData[participant["puuid"]]["units"].append(unit["character_id"])
      for augment in participant["augments"]:
        matchData[participant["puuid"]]["augments"].append(augment)

    summonerMatchData.append(matchData)

  return summonerMatchData, calls, sleepTime

In [7]:
grandmasterData = {}
calls = 0
sleepTime = 0

for ID in tqdm(summonerIDs):
    try:
      grandmasterData[ID], c, st = gatherSummonerData(ID)
      calls += c
      sleepTime += st

      if calls % 100 == 0:
        print("100 requests reached. Sleeping 1 minute.")
        time.sleep(60)
          
    except:
        print(f"{ID} did not load.")

with open("grandmasterData.json", "w") as outfile:
    json.dump(grandmasterData, outfile)

  3%|██                                                                             | 13/500 [05:58<2:50:24, 21.00s/it]

-T1WP3YFy0ZAjeE2UMXN0pi6jQSER5Tmqt2GX3p6Z1TUIdo did not load.


  3%|██▏                                                                            | 14/500 [06:00<2:05:25, 15.48s/it]

JAK9EHKMsmnXtN2PkNSqx99r3LDEs0ql2cCXZ1uIMrlKk9c did not load.


  3%|██▎                                                                            | 15/500 [06:03<1:34:11, 11.65s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████████▏| 495/500 [2:25:24<01:15, 15.10s/it]

z_GX5rz80tbJpwKTVn0srXjq4cy1dG4C6AwSb-2wSNxLd1Y did not load.


 99%|█████████████████████████████████████████████████████████████████████████████▍| 496/500 [2:25:27<00:45, 11.38s/it]

sU01hO-yB56tuw0ee77jIn1NRZ0E-D3kYoth3FBBFWZbBeA did not load.


100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [2:27:02<00:00, 17.65s/it]
